In [1]:
import sys
sys.path.append("..")

import collections
import itertools

import numpy as np

import vaalianalyysi as va

# Onko äänelläni väliä?

Tässä tarkastellaan äänien vaikuttavuutta eri kunnissa: monestako äänestäjästä tulos oli kiinni?

Lasken, että tulos oli kiinni $n$ äänestäjästä, jos paikkamäärien muuttamiseen olisi riittänyt, että $n$ äänestäjää tekee erilaisen päätöksen, eli:

* jos $n$ äänestäjää, jotka äänestivät puoluetta $A$, olisivatkin jättäneet äänestämättä;
* jos $n$ äänestäjää, jotka jättivät äänestämättä, olisivatkin äänestäneet puoluetta $A$;
* jos $n$ äänestäjää, jotka äänestivät puoluetta $A$, olisivatkin äänestäneet puoluetta $B$.

Toiset siirtymät ovat realistisempia kuin toiset (esimerkiksi siirtyminen Vihreistä Vasemmistoliittoon lienee todennäköisempää kuin Perussuomalaisista Vihreisiin), ja toiset paikkamäärien muutokset ovat vaikuttavampia kuin toiset (jos Vasemmistoliitto menettää paikan SDP:lle, on se vähemmän vaikuttavaa kuin jos se olisi menettänyt paikan Kokoomukselle). Mutta nämä ovat subjektiivisia asioita, enkä huomioi niitä tässä.

In [2]:
# Kuntavaalien 2021 tulokset
V = va.data.tulospalvelu.vaalit("KV-2021")

Seuraavat funktiot laskevat, montako paikkaa kunnissa on tarjolla, montako ääntä kukin puolue sai, ja miten paikat jakautuvat d'Hondtin mukaisesti.

In [3]:
# Montako valitaan tässä kunnassa?
def paikkoja(kunta_nro):
    df = V.tulokset_alueittain
    df = df[df.alueen_tyyppi == "K"]
    df = df[df.kunta_nro.astype(int) == int(kunta_nro)]
    return df.iloc[0].paikkoja

# Montako ääntä kukin puolue sai?
def ääniä(kunta_nro):
    df = V.tulokset_ehdokkaittain
    df = df[df.alueen_tyyppi == "K"]
    df = df[df.kunta_nro.astype(int) == int(kunta_nro)]
    df = df.groupby("puolue_lyhenne")["n"].sum().reset_index()
    return [int(n) for n in df.n.to_numpy()], [str(pl) for pl in df.puolue_lyhenne.to_numpy()]

# Montako paikkaa puolueet saavat, kun äänet ovat N ja valitaan k?
def dhondt(N, k):
    vertailuluvut = []
    for i, n in enumerate(N):
        for j in range(1, k + 1):
            vertailuluvut.append((n / j, i))

    äänet = [0] * len(N)
    for vl, i in sorted(vertailuluvut)[::-1][:k]:
        äänet[i] += 1
    return äänet

Tässä lasketaan, monestako äänestäjästä tulos on ollut kiinni.

In [4]:
# Eri tavat, jolla vaalitulos voisi muuttua dn äänellä.
def mutate(N0, k, dn):
    for i in range(len(N0)):
        N = N0[:]
        N[i] += dn
        yield N
    for i in range(len(N0)):
        N = N0[:]
        N[i] -= dn
        yield N
    for i in range(len(N0)):
        for j in range(len(N0)):
            if i == j:
                continue
            N = N0[:]
            N[i] += dn
            N[j] -= dn
            yield N

# Monestako äänestä on kiinni, että paikkamäärä muuttuu?
def kiinni(N0, k):
    d0 = dhondt(N0, k)
    for dn in itertools.count(1):
        for N in mutate(N0, k, dn):
            if dhondt(N, k) != d0:
                return dn, N

# Kuntakohtaisten tulosten laskenta

In [5]:
def report(nimi, dn, N, dN, Nt):
    if np.sum(dN) < np.sum(N):
        p = [p for a, b, p in zip(N, dN, Nt) if a != b][0]
        print(f"Kunnassa {nimi} paikkamäärät olisivat muuttuneet, jos puolueen {p} äänestäjistä {dn} olisi jättänyt äänestämättä.")
    elif np.sum(dN) > np.sum(N):
        p = [p for a, b, p in zip(N, dN, Nt) if a != b][0]
        print(f"Kunnassa {nimi} paikkamäärät olisivat muuttuneet, jos nukkuvista {dn} olisi äänestänyt puoluetta {p}.")
    else:
        pa = [p for a, b, p in zip(N, dN, Nt) if a < b][0]
        pb = [p for a, b, p in zip(N, dN, Nt) if a > b][0]
        print(f"Kunnassa {nimi} paikkamäärät olisivat muuttuneet, jos puolueen {pb} äänestäjistä {dn} olisikin äänestänyt puoluetta {pa}.")

tulos = collections.defaultdict(list)
for nro, kunta in V.kunnat.items():
    N, Nt = ääniä(nro)
    k = paikkoja(nro)
    dn, dN = kiinni(N, k)
    report(kunta.nimi_suomeksi, dn, N, dN, Nt)
    tulos[dn].append(kunta.nimi_suomeksi)

Kunnassa Helsinki paikkamäärät olisivat muuttuneet, jos puolueen LIIK äänestäjistä 84 olisikin äänestänyt puoluetta RKP.
Kunnassa Askola paikkamäärät olisivat muuttuneet, jos puolueen SDP äänestäjistä 2 olisikin äänestänyt puoluetta VIHR.
Kunnassa Espoo paikkamäärät olisivat muuttuneet, jos puolueen KD äänestäjistä 39 olisikin äänestänyt puoluetta KOK.
Kunnassa Hanko paikkamäärät olisivat muuttuneet, jos puolueen RKP äänestäjistä 6 olisikin äänestänyt puoluetta KOK.
Kunnassa Hyvinkää paikkamäärät olisivat muuttuneet, jos puolueen SDP äänestäjistä 33 olisikin äänestänyt puoluetta KESK.
Kunnassa Inkoo paikkamäärät olisivat muuttuneet, jos puolueen KOK äänestäjistä 22 olisikin äänestänyt puoluetta RKP.
Kunnassa Järvenpää paikkamäärät olisivat muuttuneet, jos puolueen Plus äänestäjistä 30 olisikin äänestänyt puoluetta VAS.
Kunnassa Karkkila paikkamäärät olisivat muuttuneet, jos puolueen KD äänestäjistä 3 olisikin äänestänyt puoluetta VIHR.
Kunnassa Kauniainen paikkamäärät olisivat muuttune

# Kunnat lukumäärittäin

In [6]:
for k in sorted(tulos.keys()):
    t = tulos[k]
    print(f"Seuraavissa {len(t)} kunnassa tulos oli {k} äänestäjästä kiinni:")
    for k in sorted(t):
        print(f"  {k}")

Seuraavissa 22 kunnassa tulos oli 1 äänestäjästä kiinni:
  Alavus
  Enontekiö
  Hankasalmi
  Ilmajoki
  Kihniö
  Kolari
  Lapinlahti
  Mänttä-Vilppula
  Nakkila
  Pelkosenniemi
  Pello
  Pietarsaari
  Pomarkku
  Pornainen
  Pukkila
  Puolanka
  Puumala
  Pöytyä
  Ruokolahti
  Taivassalo
  Tervo
  Ulvila
Seuraavissa 21 kunnassa tulos oli 2 äänestäjästä kiinni:
  Askola
  Hyrynsalmi
  Isojoki
  Kannonkoski
  Karvia
  Kauhava
  Kiuruvesi
  Koski Tl
  Laihia
  Lapinjärvi
  Lestijärvi
  Masku
  Mäntsälä
  Paimio
  Polvijärvi
  Siikalatva
  Simo
  Säkylä
  Tervola
  Vehmaa
  Vesanto
Seuraavissa 29 kunnassa tulos oli 3 äänestäjästä kiinni:
  Harjavalta
  Heinola
  Hirvensalmi
  Hämeenkyrö
  Ii
  Iisalmi
  Ilomantsi
  Joutsa
  Juupajoki
  Karkkila
  Keuruu
  Lappajärvi
  Laukaa
  Merikarvia
  Multia
  Muonio
  Nurmes
  Punkalaidun
  Reisjärvi
  Savitaipale
  Siikainen
  Siilinjärvi
  Sodankylä
  Sonkajärvi
  Sotkamo
  Teuva
  Tohmajärvi
  Uusikaarlepyy
  Äänekoski
Seuraavissa 21 kunnassa tulos